In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

In [2]:
from langchain_core.prompts import PromptTemplate

In [3]:
from langchain_core.prompts import ChatPromptTemplate

In [4]:
import requests

In [5]:
from dotenv import load_dotenv

In [7]:
from dotenv import load_dotenv
import os
env_path = r"E:\weather-agent\.env"
load_dotenv(env_path)
print("GOOGLE_API_KEY loaded:", bool(os.getenv("GOOGLE_API_KEY")))

GOOGLE_API_KEY loaded: True


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
from langchain_core.prompts import PromptTemplate

# Define a simple prompt template
prompt_template = PromptTemplate(
    input_variables=["name"],
    template="Hello, {name}! How can I help you today?"
)

# Generate the prompt by filling in the variable
formatted_prompt = prompt_template.format(name="Santu")

print(formatted_prompt)

Hello, Santu! How can I help you today?


In [11]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI

# Use one of the models from your list_models() output
chat_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # Matches your output: models/gemini-2.5-flash
    temperature=0
)

response = chat_model.invoke("What is the capital of the Nepal?")
print(response.content)

The capital of Nepal is **Kathmandu**.


In [12]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

# 1. Initialize the Gemini model instead of OpenAI
# Using 'gemini-2.5-flash' from your available models list
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

learn_template = """
I want you to act as a consultant for a AI training
Return a list of topics and why it is important to learn in given area of AI
The description should be relevant to recent advancement in AI
What are some good topics to learn in {AI_topic}
"""

prompt_template = PromptTemplate(
    input_variables=["AI_topic"],
    template=learn_template,
)

# 2. Modern LCEL Chain (Replaces legacy LLMChain)
# This pipes the prompt into the llm, then parses the output as a string
chain = prompt_template | llm | StrOutputParser()

description = "Deep learning"

# 3. Run the chain
# In LCEL, we use .invoke() instead of .run()
response = chain.invoke({"AI_topic": description})

print(response)

As an AI Training Consultant, I've observed that the landscape of Deep Learning is evolving at an unprecedented pace. To equip individuals for success in this dynamic field, training must go beyond foundational concepts to embrace the cutting-edge advancements that are shaping the industry.

Here is a list of essential Deep Learning topics, categorized for clarity, along with why each is crucial for staying relevant in today's AI environment:

---

## Essential Deep Learning Topics for Current AI Landscape

### I. Foundational Concepts & Tools (The Bedrock)

1.  **Neural Network Fundamentals & Architectures:**
    *   **Topics:** Perceptrons, Multi-Layer Perceptrons (MLPs), Activation Functions (ReLU, GELU, Swish), Loss Functions (Cross-Entropy, MSE), Optimizers (SGD, Adam, Adagrad, RMSprop, AdamW), Backpropagation.
    *   **Why it's Important:** These are the absolute building blocks. Even with highly abstracted frameworks, a deep understanding of these fundamentals is critical for d

In [14]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import Tool
# Modern import for 2025
from langchain.agents import create_agent

load_dotenv()

# 1. Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0
)

# 2. Define custom tool
def my_tool_function(query: str) -> str:
    return f"Tool response: {query}"

my_tool = Tool.from_function(
    func=my_tool_function, 
    name="simple_tool", 
    description="Use this for simple responses."
)

# 3. Setup Tools
ddg_search = DuckDuckGoSearchRun()
tools = [ddg_search, my_tool]

# 4. Create the Agent (New v1.0 Standard)
# This creates a production-ready agent without needing AgentExecutor
agent = create_agent(
    llm, 
    tools=tools, 
    system_prompt="You are a helpful assistant. Use tools for real-time info."
)

# 5. Run the agent
# Using modern .invoke() syntax
response = agent.invoke({"messages": [("user", "What's the weather like today in nepal?")]})

# Print the last message from the agent
print(response["messages"][-1].content)

[{'type': 'text', 'text': 'In Bagmati, Nepal, the current weather is sunny with a real-time temperature of 19°C.', 'extras': {'signature': 'CpoSAXLI2nyD8ZzRlIlDDyzC3metRdtugSP0UO74IhabzJ34V0o9IkOevBbuAQpoh2x7Ih8SUjyIQAb2jIp7ZPMSaydwhKAcmJuFGw/xUUbxX4EhUDu4SL8haeA9uOVlWxCl8uRfdX8Ro/vVKQmNG3K6Sc3fp6JM1R2qEtF7Yr3iuDNy5om3kXvjlOjB3O1dPe8FQxZ2VOgGiIooCcmO30xhO5OSdPMHZp0XwgIz0a9dTVEEI30nKIEQYcaWCqzqVkpfjRVouGKaOWiP9BsfynsP29SIrbxMfTpkHc6xIYShkhcTdVNmkn0gUHNL0vQpvQqky9mUlhK9CJuRHLLXrdxpIBCbtVDnq8SKUe69sbiYs5urkLq9F49Qft490/Je2WFFkm9RNmlw2KKAP9h4q/2coqDQ7loMz2/xk73E1oEEiyPBiNVRHpofbUWdSlglWZZKUT5abeuRjQUEJNRoDsQjAFCrflJV4UuaulmrHg+cIyOy5y+8gSQoq7QJr/a5aOqEx1gWnf8ZYKqFWuitONt1NeRG+fo/xYA13HsyNcnV0dn9/BV13RrpXHLvBcHbCxsI3rCsziY86gRj0wW9t0aOaLOwMSLPwsxgIy239VsLakvjRaz8eWlNPsv9ULtwfXm59UPl1+VnqhwUu1FiQ+UiNZvihHIcOM7gQm0ChbL9bmS+GWq4R2mf9brtHuxqTAKJnp/h97kn3K2MAfnG3KGlj6cpgM3XJ3Je6Wv53GPabn5W86f9mxL4TksOpwA0On+dN0+o8Zff0zf5DH6KCinG0SAwm5Zt7g8jx2+R69BAlbrr/7TR5qyktNPhryw4B5HR/OvkysRru5qzDtvC+JzjSo+7

In [16]:
!pip show langchain langchain-community duckduckgo-search python-dotenv requests ddgs langchain-google-genai google-generativeai

Name: langchain
Version: 1.2.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: E:\weather-agent-implement\myenv\Lib\site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
---
Name: langchain-community
Version: 0.4.1
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: E:\weather-agent-implement\myenv\Lib\site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain-classic, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: duckduckgo_search
Version: 8.1.1
Summary: Search for words, documents, images, news, maps and text translation using the DuckDuckGo.com search engine.
Home-page: https://github.com/deedy5/duckduckgo_search
Author: deedy5
Author-email: 
License: MIT License
Location: E:\weather-agent-implement\myenv\Lib\site-package